In [1]:
import tensorflow as tf
print ("TensorFlow version: " + tf.__version__)
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import numpy as np
from keras import layers
from keras.models import Model
from keras.models import load_model
from keras.layers.core import Dense, Dropout, Activation, Reshape
from keras import callbacks


TensorFlow version: 2.2.0


In [2]:
xTemp = np.zeros((1060, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\TrainingA_1-1060.npy", 'rb') as f1:
        xTemp = np.load(f1)
        

In [3]:
yTemp = np.zeros((1060))
for i in range (1060):
    if (i%2==0):
        yTemp[i] = 0
    else:
        yTemp[i] = 1
        

In [4]:
x = np.zeros((1060, 6, 340, 444))
y = np.zeros((1060))

permutations = np.random.permutation(1060)
for i in range (1060):
    x[i] = xTemp[permutations[i]]
    y[i] = yTemp[permutations[i]]


In [5]:
x = np.reshape(x, (1060, 340, 444, 6))
y = y.astype(int)
img_shape = (340, 444, 6)

In [6]:
validation = np.zeros((212, 6, 340, 444))
with open("C:\\Users\\yangz\\Desktop\\FireProjectBatchDownloadNew\\DetectionDatasetFinal\\Validation_1908-2120.npy", 'rb') as f1:
        validation = np.load(f1)

In [7]:
validation = np.reshape(validation, (212, 340, 444, 6))
yValidation = np.zeros((212))
for i in range (212):
    if (i%2==0):
        yValidation[i] = 0
    else:
        yValidation[i] = 1

In [8]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import roc_auc_score
import sys

space = {
            'units1': hp.choice('units1', [16, 32, 64, 128, 256]),
            'units2': hp.choice('units2', [16, 32, 64, 128, 256]),
            'units3': hp.choice('units3', [16, 32, 64, 128, 256]),
            'units4': hp.choice('units4', [16, 32, 64, 128, 256]),
            'units5': hp.choice('units5', [16, 32, 64, 128, 256]),
            'units6': hp.choice('units6', [16, 32, 64, 128, 256]),
    
            'poolSize1': hp.choice('poolSize1', [3, 5]),
            'poolSize2': hp.choice('poolSize2', [3, 5]),
            'poolSize3': hp.choice('poolSize3', [3, 5]),
            'poolSize4': hp.choice('poolSize4', [3, 5]),
            'poolSize5': hp.choice('poolSize5', [3, 5]),

            'dropout1': hp.uniform('dropout1', .20,.70),

            'batchSize' : hp.choice('batchSize', [4, 8, 16, 32]),

            'nbEpochs' : hp.choice('nbEpochs', [9, 10, 11, 12]),
    
            'learnRate1' : hp.choice('learnRate1', [1e-3, 1e-4, 1e-5]), 
        }


In [9]:
from tqdm.keras import TqdmCallback

def f_nn(params):
    img = layers.Input(shape=img_shape)

    conv1 = layers.Conv2D(params['units1'], (params['poolSize1'], params['poolSize1']), padding='same', activation='relu', strides=5)(img)
    bn1 = layers.BatchNormalization()(conv1)
    maxPooling1 = layers.MaxPooling2D(padding='same')(bn1)

    conv2 = layers.Conv2D(params['units2'], (params['poolSize2'], params['poolSize2']), padding='same', activation='elu', strides=5)(maxPooling1)
    bn2 = layers.BatchNormalization()(conv2)
    maxPooling2 = layers.MaxPooling2D(padding='same')(bn2)

    conv3 = layers.Conv2D(params['units3'], (params['poolSize3'], params['poolSize3']), padding='same', activation='elu', strides=3)(maxPooling2)
    bn3 = layers.BatchNormalization()(conv3)
    maxPooling3 = layers.MaxPooling2D(padding='same')(bn3)

    conv4 = layers.Conv2D(params['units4'], (params['poolSize4'], params['poolSize4']), padding='same', activation='elu', strides=3)(maxPooling3)
    bn4 = layers.BatchNormalization()(conv4)
    maxPooling4 = layers.MaxPooling2D(padding='same')(bn4)

    conv5 = layers.Conv2D(params['units5'], (params['poolSize5'], params['poolSize5']), padding='same', activation='elu', strides=3)(maxPooling4)
    maxPooling5 = layers.MaxPooling2D(padding='same')(conv5)

    flatten = layers.Flatten()(maxPooling5)

    dense1 = layers.Dense(params['units6'], activation = "relu")(flatten)
    drop2 = layers.Dropout(params['dropout1'])(dense1)
    res = layers.Dense(1, activation='sigmoid')(drop2)

    model = Model(img, res)
    opt = tf.keras.optimizers.RMSprop(learning_rate=params['learnRate1'])
    model.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    model.fit(x, y, batch_size = int(params['batchSize']), epochs = params['nbEpochs'], verbose=0, callbacks=[TqdmCallback(verbose=1)])
    
    yPred = model.predict(validation)
    yPred[yPred>=0.5] = 1
    yPred[yPred<0.5] = 0
    yPred = yPred.astype(int)
    yPred = yPred.flatten()
    
    correct = 0
    for i in range (212):
        if (yPred[i]==yValidation[i]):
            correct+=1
    acc = correct/212
    
    print('Accuracy:', acc)
    sys.stdout.flush() 
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=20, trials=trials)
print('best: ', best)

  0%|                                                                           | 0/20 [00:00<?, ?trial/s, best loss=?]

Accuracy:                                                                                                              
0.75                                                                                                                   
  5%|███                                                          | 1/20 [03:02<57:40, 182.14s/trial, best loss: -0.75]

Accuracy:                                                                                                              
0.7783018867924528                                                                                                     
 10%|████▌                                        | 2/20 [11:39<1:24:47, 282.63s/trial, best loss: -0.7783018867924528]

Accuracy:                                                                                                              
0.7688679245283019                                                                                                     
 15%|██████▊                                      | 3/20 [20:24<1:40:43, 355.48s/trial, best loss: -0.7783018867924528]

Accuracy:                                                                                                              
0.6650943396226415                                                                                                     
 20%|█████████                                    | 4/20 [23:10<1:19:38, 298.63s/trial, best loss: -0.7783018867924528]

...

Accuracy:                                                                                                              
0.6132075471698113                                                                                                     
 80%|███████████████████████████████████▏        | 16/20 [1:15:30<15:16, 229.03s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.7169811320754716                                                                                                     
 85%|█████████████████████████████████████▍      | 17/20 [1:20:05<12:08, 242.93s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.7452830188679245                                                                                                     
 90%|███████████████████████████████████████▌    | 18/20 [1:25:54<09:09, 274.66s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.7075471698113207                                                                                                     
 95%|█████████████████████████████████████████▊  | 19/20 [1:30:10<04:29, 269.17s/trial, best loss: -0.8066037735849056]

Accuracy:                                                                                                              
0.7264150943396226                                                                                                     
100%|████████████████████████████████████████████| 20/20 [1:33:54<00:00, 281.73s/trial, best loss: -0.8066037735849056]
best:  {'batchSize': 0, 'dropout1': 0.5420313649456072, 'learnRate1': 2, 'nbEpochs': 3, 'poolSize1': 1, 'poolSize2': 1, 'poolSize3': 1, 'poolSize4': 0, 'poolSize5': 0, 'units1': 4, 'units2': 2, 'units3': 2, 'units4': 4, 'units5': 1, 'units6': 4}


In [ ]:
#Acc1: 0.9316 / 0.6650943396226415